# AutogradTensor Development

We have an issue with backprop in PyTorch. When you call `.backward()` on a torch.Tensor it immediately drops down into C++ code. However, we need to keep track of all operations in Python for use in calculating sensitivity as well as other things. We need to override `.backward()` such that we go through the backpropagation chain in Python instead of the C++ layer.

The idea here is we create a `BackpropTensor` that overrides `.backward()`. Maybe a different name, like `GradientTensor` since it should only be used on tensors where we need gradients. We can discuss naming later I suppose. Anyway, this new tensor should override `.backward()`...

In [2]:
import torch

In [ ]:
for grad, grad_fn in zip(grads, z.grad_fn.next_functions):
    grad_fn[0](grad)

In [11]:
grad_fn = GradAdd(1, 2)
grad_fn(torch.ones(1,2))

tensor([[1., 1.]])

In [70]:
class GradAdd:
    def __init__(self, *args):
        self.next_functions = []
        
    def __call__(self, grad):
        return grad # * 1

In [84]:
class Accumulate:
    def __init__(self, tensor):
        self.tensor = tensor
        self.next_functions = []
        
    def __call__(self, grad):
        if self.tensor.grad:
            self.tensor.grad += grad
        else:
            self.tensor.grad = grad + 0

In [85]:
def backwards_grad(grad_fn, in_grad=None):
    back_grad = grad_fn(in_grad)
    for next_grad_fn in grad_fn.next_functions:
        print(next_grad_fn)
        backwards_grad(next_grad_fn, back_grad)
    
class Tensor:
    def __init__(self, data, requires_grad=True):
        self.grad = None
        self.grad_fn = None
        self.child = data
        self.requires_grad = requires_grad
    
    def backward(self, grad=None):
        if grad is None:
            grad = torch.ones_like(self.child)
        backwards_grad(self.grad_fn, grad)
        
    def add(self, other):
        result = Tensor(self.child + other.child)
        
        grad_fn = GradAdd()
        grad_fn.next_functions = (Accumulate(self) if self.grad_fn is None else self.grad_fn, 
                                  Accumulate(other) if self.grad_fn is None else self.grad_fn)
        result.grad_fn = grad_fn
        
        return result

In [86]:
x = Tensor(torch.randn(3), requires_grad=True)
y = Tensor(torch.randn(3))

In [87]:
z = x.add(y)

In [88]:
x.grad

In [89]:
z.backward()

In [90]:
x.grad

tensor([1., 1., 1.])

In [34]:
w.grad_fn.next_functions

In [35]:
x = torch.rand((1, 3), requires_grad=True)
y = torch.rand((3, 1), requires_grad=True)

In [38]:
z.grad_fn.next_functions[0][0]

((<MmBackward at 0x122deaf60>, 0),)

In [36]:
z = torch.mm(x, y).mean()

In [6]:
z.grad_fn.next_functions

((<AccumulateGrad at 0x122da5fd0>, 0), (<AccumulateGrad at 0x122d2e048>, 0))

In [6]:
for grad, grad_fn in zip(grads, z.grad_fn.next_functions):
    grad_fn[0](grad)

In [13]:
torch.autograd.

<function torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False)>

In [52]:
# gradient of z w.r.t. y
grad_z_y = z.grad_fn(torch.ones(1))

In [56]:
# gradient of y w.r.t. x
grad_y_x = y.grad_fn(grad_z_y)

In [76]:
x.grad

In [77]:
def backwards_grad(grad_fn, in_grad=None):
    back_grad = grad_fn(in_grad)
    for next_grad_fn, _ in grad_fn.next_functions:
        print(next_grad_fn)
        backwards_grad(next_grad_fn, back_grad)

def backward(tensor):
    backwards_grad(tensor.grad_fn, torch.ones_like(tensor))

In [80]:
backward(z)

In [81]:
x.grad

tensor([1.3333, 2.6667, 4.0000], grad_fn=<AddBackward0>)

In [82]:
x

tensor([1., 2., 3.], requires_grad=True)

In [83]:
y

tensor([1., 4., 9.], grad_fn=<PowBackward0>)

In [88]:
w = torch.cat((x, y)).mean()

In [1]:
import sys
sys.path.append('/Users/mat/Projects/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
a = AutogradTensor().on(torch.tensor([3, 2., 0]))
b = AutogradTensor().on(torch.tensor([1, 2., 3]))

In [3]:
c = a + b

In [4]:
c.grad_fn(torch.ones(3))

(tensor([1., 1., 1.]), tensor([1., 1., 1.]))

In [5]:
c.backward()

In [6]:
# need to fix this, should show the gradient
a.grad

In [7]:
a.child.grad

tensor([1., 1., 1.])

In [8]:
b.child.grad

tensor([1., 1., 1.])

In [9]:
d = AutogradTensor().on(torch.rand(3))
e = AutogradTensor().on(torch.rand(3))

In [10]:
f = d.sin() * e.sin()

In [11]:
f.grad_fn

MulBackward

In [12]:
f.grad_fn.next_functions

(SinBackward, SinBackward)

In [13]:
f.backward()

In [15]:
e.child.grad

tensor([0.4016, 0.0379, 0.1268])